In [1]:
#pool functions
def calculateInvariant_b(A, pool, roundUp):
    n = len(pool)
    S = 0
    for i in range(0, n):
        if (not(isinstance(pool[i], int))):
            raise TypeError('Not int in pool')
        S = S + pool[i]
    prevD = 0
    D = S
    An = mul(A, pow(n, n))
    for i in range(0, 255):
        P_D = pool[0]*n
        for j in range (1, n):
                P_D = div(mul(mul(P_D, pool[j]), n), D, roundUp)
        prevD = D
        D = div(
            add(mul(mul(n, D), D), mul(mul(An, S), P_D)
            ),
            add(mul(n + 1, D), mul(An - 1, P_D)
            ),
            roundUp
            )
        if (D > prevD):
            if (D - prevD <= 1):
                return D
        elif (prevD - D <= 1):
            return D

    raise NoConvergenceException('D didn\'t converge, error = ', abs(D - prevD))
    
def calculateInvariant(A, pool):
    n = len(pool)
    S = sum(pool)
    P = 1
    for i in range(0, n):
        P = P*pool[i]
    t = pow(n, n)
    D = S
    T = A*t
    
    for i in range(0, 255):
        prevD = D
        K = pow(D, n)/(t*P)
        D = (n*K*D + T*S)/(T + (n+1)*K - 1)
        if (D > prevD):
            if (D - prevD <= 1):
                return D
        elif (prevD - D <= 1):
            return D
    raise NoConvergenceException('D didn\'t converge')
        


def getTokenBalanceGivenInvariantAndAllOtherBalances_b(A, pool, D, tokenindex):
    n = len(pool)
    S = pool[0]
    P_D = pool[0]*n
    for j in range(1, n):
        P_D = divDown(mul(mul(P_D, pool[j]), n), D)
        S = add(S, pool[j])
    S = sub(S, pool[tokenindex])
    D2 = mul(D, D)
    An = int(A*pow(n, n))
    
    c = mul(
        divUp(D2, mul(An, P_D)),
        pool[tokenindex]
    )
    
    b = add(S,divDown(D, An))
    
    prevTokenBalance = 0
    
    tokenBalance = divUp(add(D2, c), add(D, b))
    
    
    for i in range(0, 255):
        prevTokenBalance = tokenBalance
        
        tokenBalance = divUp(
            add(mul(tokenBalance, tokenBalance), c),
            sub(add(mul(tokenBalance, 2), b), D)
        )
        if (tokenBalance > prevTokenBalance):
            if (tokenBalance - prevTokenBalance <= 1000):
                return tokenBalance
        elif (prevTokenBalance - tokenBalance <= 1000):
            return tokenBalance
        
    raise NoConvergenceException('balance didn\'t converge, error = ', abs(tokenBalance - prevTokenBalance))
        
def getTokenBalanceGivenInvariantAndAllOtherBalances(A, pool, D, tokenindex):
    n = len(pool)
    S = 0
    P = 1
    for i in range(0, n):
        if (i != tokenindex):
            S = S + pool[i]
            P = P*pool[i]
    t = pow(n, n) #N
    G = A*t #A
    K = pow(D, n+1)/(P*t)
    L = G*S + D - G*D
    x = D
    for i in range(0, 255):
        x = (pow(x, 2)*G + K)/(L + 2*x*G)
    return x
    
        
def calcOutGivenIn_b(A, pool, tokenin, tokenout, amountin):
    D = calculateInvariant_b(A, pool, True)
    pool[tokenin] = add(pool[tokenin], amountin)
    finalBalanceOut = getTokenBalanceGivenInvariantAndAllOtherBalances_b(A, pool, D, tokenout)
    pool[tokenin] = pool[tokenin] - amountin
    return sub(sub(pool[tokenout], finalBalanceOut), 1)

def calcOutGivenIn(A, pool, tokenin, tokenout, amountin):
    D = calculateInvariant(A, pool)
    pool[tokenin] = pool[tokenin] + amountin
    xout = getTokenBalanceGivenInvariantAndAllOtherBalances(A, pool, D, tokenout)
    pool[tokenin] = pool[tokenin] - amountin
    return pool[tokenout] - xout


def calcInGivenOut_b(A, pool, tokenin, tokenout, amountout):
    D = calculateInvariant_b(A, pool, True)
    pool[tokenout] = sub(pool[tokenout], amountout)
    finalBalanceIn = getTokenBalanceGivenInvariantAndAllOtherBalances_b(A, pool, D, tokenin)
    pool[tokenout] = pool[tokenout] + amountout
    return add(sub(finalBalanceIn, pool[tokenin]), 1)

def calcInGivenOut(A, pool_, tokenin, tokenout, amountout):
    pool = copy.deepcopy(pool_)
    D = calculateInvariant(A, pool) 
    pool[tokenout] = pool[tokenout] - amountout
    finalBalanceIn = getTokenBalanceGivenInvariantAndAllOtherBalances(A, pool, D, tokenin)
    return finalBalanceIn - pool[tokenin]

def calcBptOutGivenExactTokensIn(A, pool, amountsin, bpttotalsupply, swapfeepercentage):
    n = len(pool)
    n2 = len(amountsin)
    if (n != n2):
        print('pool and amounts_in have different dimensions: ', n, n2)
        return
    S = pool[0]
    for j in range(1, n):
        S = add(S, pool[j])
    balanceRatiosWithFee = [0 for i in range(n)]
    invariantRatioWithFees = 0
    for i in range(0, n):
        currentWeight = divDown_f(pool[i], S)
        balanceRatiosWithFee[i] = divDown_f(add(pool[i], amountsin[i]), pool[i])
        invariantRatioWithFees = add(invariantRatioWithFees, mulDown_f(balanceRatiosWithFee[i], currentWeight))
    print(balanceRatiosWithFee)
    print(invariantRatioWithFees)
        
    newpool = [0 for i in range(n)]
    for i in range(0, n):
        if (balanceRatiosWithFee[i] > invariantRatioWithFees):
            nonTaxableAmount = mulDown_f(pool[i], sub(invariantRatioWithFees, ONE))
            taxableAmount = sub(amountsin[i], nonTaxableAmount)
            amountInWithoutFee = add(nonTaxableAmount, mulDown_f(taxableAmount, ONE - swapfeepercentage))
        else:
            amountInWithoutFee = amountsin[i]
            
        newpool[i] = add(pool[i], amountInWithoutFee)
        
    currentD = calculateInvariant_b(A, pool, True)
    newD = calculateInvariant_b(A, newpool, False)
    invariantRatio = divDown_f(newD, currentD)
    if (invariantRatio > 1):
        return mulDown_f(bpttotalsupply, (invariantRatio - ONE))
    else:
        return 0
    
def calcTokenInGivenExactBptOut(A, pool, token, bptamountout, bpttotalsupply):
    D = calculateInvariant(A, pool)
    newD = (bpttotalsupply + bptamountout)/(bpttotalsupply)*D
    newBalanceToken = getTokenBalanceGivenInvariantAndAllOtherBalances(A, pool, newD, token)
    amountInWithoutFee = newBalanceToken - pool[token]
    S = sum[pool]
    currentWeight = pool[token]/S
    taxablePercentage = 1 - currentWeight
    taxableAmount = amountInWithoutFee*taxablePercentage
    nonTaxableAmount = amountInWithoutFee - taxableAmount
    
    return nonTaxableAmount + taxableAmount/(1-swapfeepercentage)

def calcBptInGivenExactTokensOut(A, pool, amountsout, bpttotalsupply):
    S = sum(pool)
    balanceRatiosWithoutFee = []
    invariantRatioWithoutFees = 0
    for i in range(0, n):
        currentWeight = pool[i]/S
        balanceRatiosWithoutFee[i] = (pool[i]-amountsout[i])/pool[i]
        invariantRatioWithoutFees = invariantRatioWithoutFees + balanceRatiosWithoutFee[i]*currentWeight
        
    newpool = []
    for i in range(0, n):
        if (invariantRatioWithoutFees > balanceRatiosWithoutFee[i]):
            nonTaxableAmount = pool[i]*(1-invariantRatioWithoutFees)
            taxableAmount = amountsout[i] - nonTaxableAmount
            amountOutWithFee = nonTaxableAmount + taxableAmount/(1-swapfeepercentage)
        else:
            amountOutWithFee = amountsout[i]
        newpool[i] = pool[i] - amountOutWithFee
        
    D = calculateInvariant(A, pool)
    newD = calculateInvariant(A, newpool)
    invariantRatio = newD/D
    return bpttotalsupply*(1-invariantRatio)

def calcTokenOutGivenExactBptIn(A, pool, token, bptAmountIn, bptTotalSupply, swapfeepercentage):
    D = calculateInvariant(A, pool)
    newD = (bptTotalSupply - bptAmountIn)/bptTotalSupply*D
    
    newBalanceToken = getTokenBalanceGivenInvariantAndAllOtherBalances(A, pool, newD, token)
    amountOutWithoutFee = pool[token] - newBalanceToken
    S = sum[pool]
    currentWeight = pool[token]/S
    taxablePercentage = (1-currentWeight)
    taxableAmount = amountOutWithoutFee*taxablePercentage
    nonTaxableAmount = amountOutWithoutFee - taxableAmount
    
    return nonTaxableAmount + taxableAmount*(1-swapfeepercentage)

def calcTokensOutGivenExactBptIn(pool, bptAmountIn, bptTotalSupply):
    bptRatio = bptAmountIn/bptTotalSupply
    
    amountsOut = []
    for i in range(0, n):
        amountsOut[i] = pool[i]*bptRatio
        
    return amountsOut

def calcDueTokenProtocolSwapFeeAmount(A, pool, lastD, token, protocolSwapFeePercentage):
    finalBalanceFeeToken = getTokenBalanceGivenInvariantAndAllOtherBalances(A, pool, lastD, token)
    
    if (pool[token] <= finalBalanceFeeToken):
        return 0
    
    accumulatedTokenSwapFees = pool[token] - finalBalanceFeeToken

    
return accumulatedTokenSwapFees*protocolSwapFeePercentage/(1)

In [ ]:
#Pool - 50% LUSD, 50% BPT (USDC, USDT, DAI)


pool = np.array([])
